In [3]:
import numpy as np
import pickle
import os
from os.path import join

In [6]:
folder_path = '/home/macul/libraries/mk_utils/tf_spoof/eval_rst'
#file_name = ['train_4_0.pkl', 'train_17_0.pkl', 'train_18_4.pkl', 'train_28_1.pkl', 'train_38_9.pkl', 'train_40_2.pkl', 'train_39_3.pkl', 'train_37_8.pkl', 'train_35_2.pkl']
#file_name = ['train_38_9.pkl', 'train_41_1.pkl', 'train_42_6.pkl', 'train_37_8.pkl', 'train_36_2.pkl', 'train_35_2.pkl']
file_name = ['train_43_6.pkl',]
th = [0.6, 0.65, 0.7, 0.75, 0.8]
tf_f = [0.7, 0.75, 0.8, 0.85, 0.9]

In [7]:
rst_dic = {}
for fn in file_name:
    print(fn)
    print('')
    print('')
    
    with open(join(folder_path, fn),'rb') as f:
        rst = pickle.load(f)
        tmp_dic = {}
        
        for t in th:
            for tf in tf_f:
                true_true = 0 # gd_pred
                true_false = 0
                false_true = 0
                false_false = 0
                total = 0
                for cat in ['real','image_attack','video_attack']:
                    for k in rst[cat].keys():                    
                        data = rst[cat][k].copy()
                        total += data.shape[0]

                        tmp = np.sum(data[:,1:], axis=1)
                        data[:,1] = tmp
                        data = data[:,0:2]
                        data_real = data[np.argmax(data,axis=1)==0,:]
                        data_fake = data[np.argmax(data,axis=1)!=0,:]

                        real_rst = np.sum(data_real[:,0]>t)
                        fake_rst = np.sum(data_fake[:,1]>tf)
                        
                        #print(t, tf, k, real_rst, fake_rst)
                        
                        if cat=='real':
                            true_true += real_rst
                            true_false += fake_rst
                        else:
                            false_true += real_rst
                            false_false += fake_rst
                #print(t, tf, k, true_true, true_false, false_true, false_false)
                tmp_dic['{},{}'.format(t,tf)] = {'total': total,
                                                 'true_true': true_true,
                                                 'true_false': true_false,
                                                 'false_true': false_true,
                                                 'false_false': false_false,
                                                 'detection_rate': 1.0*(true_true+true_false+false_true+false_false)/total,
                                                 'recall': 1.0*true_true/(true_true+true_false),
                                                 'precision': 1.0*true_true/(true_true+false_true),
                                                 'accuracy': 1.0*(true_true+false_false)/(true_true+true_false+false_true+false_false),
                                                 'FPR': 1.0*false_true/(false_true+false_false)}
        
        for tk in tmp_dic.keys():
            print(tk)
            print(tmp_dic[tk])
            print('')
            '''
            print('detection_rate', tmp_dic[tk]['detection_rate'])
            print('recall', tmp_dic[tk]['recall'])
            print('precision', tmp_dic[tk]['precision'])
            print('accuracy', tmp_dic[tk]['accuracy'])
            print('FPR', tmp_dic[tk]['FPR'])
            '''
        rst_dic[fn] = tmp_dic
                    

train_43_6.pkl


0.75,0.8
{'detection_rate': 0.5255896583005746, 'total': 52912, 'FPR': 0.36892602909316, 'true_true': 23367, 'recall': 0.9506896130843403, 'false_true': 1192, 'precision': 0.9514638218168492, 'accuracy': 0.9135562747213233, 'false_false': 2039, 'true_false': 1212}

0.75,0.75
{'detection_rate': 0.5660719685515573, 'total': 52912, 'FPR': 0.2571182053494392, 'true_true': 23367, 'recall': 0.9230131142360563, 'false_true': 1192, 'precision': 0.9514638218168492, 'accuracy': 0.8951322115384616, 'false_false': 3444, 'true_false': 1949}

0.65,0.8
{'detection_rate': 0.6398737526459026, 'total': 52912, 'FPR': 0.5540244969378828, 'true_true': 28073, 'recall': 0.9586136247225542, 'false_true': 2533, 'precision': 0.9172384499771287, 'accuracy': 0.8893877189355229, 'false_false': 2039, 'true_false': 1212}

0.8,0.8
{'detection_rate': 0.44940656183852434, 'total': 52912, 'FPR': 0.25036764705882353, 'true_true': 19847, 'recall': 0.9424474096585783, 'false_true': 681, 'precision': 0.9668

In [ ]:
rst_dic = {}
for fn in file_name:
    print(fn)
    real_dic = {}
    image_attack_dic = {}
    video_attack_dic = {}
    
    with open(join(folder_path, fn),'rb') as f:
        rst = pickle.load(f)
        for k in rst['real'].keys():
            real = rst['real'][k]
            tmp_dic = {}
            tmp_dic['total'] = real.shape[0]
            tmp = np.sum(real[:,1:], axis=1)
            real[:,1] = tmp
            real = real[:,0:2]
            real_real = real[np.argmax(real,axis=1)==0,:]
            real_fake = real[np.argmax(real,axis=1)!=0,:]
            
            for t in th:
                for tf in tf_f:
                    real_rst = np.sum(real_real[:,0]>t)                    
                    fake_rst = np.sum(real_real[:,1]>tf) + np.sum(real_fake[:,1]>tf)
                    tmp_dic['{},{}'.format(t,tf)] = [1.0*(real_rst+fake_rst)/tmp_dic['total'], 
                                                     1.0*real_rst/(real_rst+fake_rst),
                                                     1.0*fake_rst/(real_rst+fake_rst)]
            real_dic[k] = tmp_dic
            #print(tmp_dic)
            
        for k in rst['image_attack'].keys():
            print(k)
            image_attack = rst['image_attack'][k]
            tmp_dic = {}
            tmp_dic['total'] = image_attack.shape[0]
            tmp = np.sum(image_attack[:,1:], axis=1)
            image_attack[:,1] = tmp
            image_attack = image_attack[:,0:2]
            image_attack_real = image_attack[np.argmax(image_attack,axis=1)==0,:]
            image_attack_fake = image_attack[np.argmax(image_attack,axis=1)!=0,:]
            
            for t in th:
                for tf in tf_f:
                    real_rst = np.sum(image_attack_real[:,0]>t)                    
                    fake_rst = np.sum(image_attack_real[:,1]>tf) + np.sum(image_attack_fake[:,1]>tf)
                    tmp_dic['{},{}'.format(t,tf)] = [tmp_dic['total'], real_rst, fake_rst, 
                                                     1.0*(real_rst+fake_rst)/tmp_dic['total'], 
                                                     1.0*real_rst/(real_rst+fake_rst),
                                                     1.0*fake_rst/(real_rst+fake_rst)]
            image_attack_dic[k] = tmp_dic
            print(tmp_dic)
            
        for k in rst['video_attack'].keys():
            print(k)
            video_attack = rst['video_attack'][k]
            tmp_dic = {}
            tmp_dic['total'] = video_attack.shape[0]
            tmp = np.sum(video_attack[:,1:], axis=1)
            video_attack[:,1] = tmp
            video_attack = video_attack[:,0:2]
            video_attack_real = video_attack[np.argmax(video_attack,axis=1)==0,:]
            video_attack_fake = video_attack[np.argmax(video_attack,axis=1)!=0,:]
            
            for t in th:
                for tf in tf_f:
                    real_rst = np.sum(video_attack_real[:,0]>t)                    
                    fake_rst = np.sum(video_attack_real[:,1]>tf) + np.sum(video_attack_fake[:,1]>tf)
                    tmp_dic['{},{}'.format(t,tf)] = [tmp_dic['total'], real_rst, fake_rst, 
                                                     1.0*(real_rst+fake_rst)/tmp_dic['total'], 
                                                     1.0*real_rst/(real_rst+fake_rst),
                                                     1.0*fake_rst/(real_rst+fake_rst)]
            video_attack_dic[k] = tmp_dic
            #print(tmp_dic)
                    

In [ ]:
np.multiply([True,False,True],[True,False,False])